In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re 
from tqdm import tqdm_notebook as tqdm

In [28]:
#データフレームの扱い。カラムを増やす。
pd.options.display.notebook_repr_html = True
pd.options.display.max_colwidth = 1000 #最大文字数の変更
pd.options.display.max_rows = 1000 #最大行数の変更

In [16]:
#J1各チームの日程・結果にアクセスするためのurlリスト。
html_all = urlopen("https://soccer.yahoo.co.jp/jleague/teams/j1")
soup_all = BeautifulSoup(html_all, "html.parser")
team_url = soup_all.find_all(href = re.compile("^/jleague/teams/schedule"))
team_url_list = []
team_name_list= []
for a in tqdm(team_url):
    team_url_list.append("https://soccer.yahoo.co.jp" + a.get("href"))
    html = urlopen("https://soccer.yahoo.co.jp" + a.get("href"))
    soup = BeautifulSoup(html, "html.parser")
    team_name = str(soup.find_all(class_ = "name"))
    team_name = re.search(pattern, team_name)
    team_name_list.append(team_name.group()[1:-1])
print(*team_name_list)


北海道コンサドーレ札幌 ベガルタ仙台 鹿島アントラーズ 浦和レッズ FC東京 川崎フロンターレ 横浜F・マリノス 湘南ベルマーレ 松本山雅FC 清水エスパルス ジュビロ磐田 名古屋グランパス ガンバ大阪 セレッソ大阪 ヴィッセル神戸 サンフレッチェ広島 サガン鳥栖 大分トリニータ


In [6]:
def score(soup):
    #各試合のスコアのTag
    score_tag = soup.find_all(href =  re.compile("^/jleague/game/score/"))
    #Tagからスコアをget
    pattern = "\d-\d"
    score_list = str(score_tag).replace("\xa0","")
    score_list = re.findall(pattern, score_list)
    return(score_tag, score_list)

In [17]:
#結果を格納しているタグを発見してテキストにする。
def results(soup):
    results = soup.find_all(class_="textC")
    #勝敗だけ抽出。
    results_list = []
    for v in results:
        if (v.text == "△" or v.text == "●" or v.text == "○" ):
            results_list.append(v.text)
    #リストの勝敗を数値に変換。
    results_list = ["2" if "△" in s  else s for s in results_list]
    results_list = ["1" if "○" in s  else s for s in results_list]
    results_list = ["0" if "●" in s  else s for s in results_list]
    return(results_list)

In [18]:
def home_review(score_tag, teamname):
    #Home/Away
    home_away_url = []
    home_away_list = []
    #review
    match_review = []
    #試合詳細のURLをリスト化
    for a in score_tag:
        home_away_url.append("https://soccer.yahoo.co.jp"  + a.get("href"))
    #リスト化されたURLからHOME/AWAY情報を抽出
    for i in tqdm(home_away_url, teamname):
        home_away_html = urlopen(i)
        home_away_soup = BeautifulSoup(home_away_html, "html.parser")
        if str(home_away_soup.title.contents[0]).split()[-7] == teamname :
            home_away_list.append(1)
        else:
            home_away_list.append(0)
        review_test = home_away_soup.find("p",class_="", text=re.compile("。")).text
        pattern = re.compile("掲載")
        matchObj = pattern.search(review_test)
    #掲載　という文字が入っていたらレビューではないので空にする。
        if matchObj:
            match_review.append("Nah")
        else:
            match_review.append(review_test)
    return(home_away_url, home_away_list, match_review)

In [51]:
###xに入るのは数字
###team x の情報を集める関数。
def gather_information(x):
    #各チームにアクセス。
    html = urlopen(team_url_list[x])
    soup = BeautifulSoup(html, "html.parser")
    #チーム名
    teamname = team_name_list[x]   
    df_score = score(soup)
    score_tag = df_score[0]
    df_results = results(soup)
    df_home_review = home_review(score_tag, teamname)
    #DataFrame→CSV にして出力
    df = pd.DataFrame({"results" : df_results,
                                     "home" : df_home_review[1],
                                     "score" : df_score[1],
                                       "match review" : df_home_review[2]
                                  })
    df.to_csv("/Users/yamadaikuya/gitrepos/toto/trial/"+str(x)+".csv",sep=",")

In [52]:
gather_information(0)

In [53]:
#全てのチームに関して情報を集める。
def gather_all_information():
    for i in tqdm(range(len(team_url_list)),"Overall Progress"):
        gather_information(i)

In [54]:
gather_all_information()

In [61]:
#csvからdfに戻す。
def show_dataframe(i):
    df_return = pd.read_csv("/Users/gitrepos/toto/trial/"+str(i)+".csv", 
                      index_col=[0])
    df_return_latest = df_return[-1:] #最新のが見たいときはこっち
    return(display(df_return))

In [64]:
print(team_name_list[0])
df_return = pd.read_csv("/Users/yamadaikuya/gitrepos/toto/trial/0.csv", 
                  index_col=[0])
df_return

北海道コンサドーレ札幌


,results,home,score,match review
0,0,0,2-0,昨季飛躍を果たした両チームの対戦はホームの湘南に軍配が上がった。ボールを支配する札幌に押し込まれる展開が続いたが、それでも運動量で相手を圧倒する「湘南スタイル」は今季も健在。時間帯が深まるにつれて徐々に流れを引き寄せると、「復帰戦」の武富が気迫のこもった2得点を挙げる。BMWスを歓喜の渦に巻き込み、3年連続の開幕戦勝利に大きく貢献した。対する札幌は攻撃陣がかみ合わず、無念の完封負け。前線ではチャナティップの単騎突破に頼るシーンが目立ち、Aロペスや鈴木ら新戦力の一刻も早いフィットが求められる。
1,1,0,0-2,開始早々に出はなをくじかれた浦和。これで相手に勢いと自信を与えてしまった。ボールを持った際は札幌の素早いプレスに自由を奪われ、またチャナティップを中心とした華麗なパスワークで何度もゴールを脅かされる展開に。シュートミスや西川のセーブに救われて早い時間帯での大量失点は回避したものの、後半も大勢は変わらず。終盤は山中や途中出場のAナバウトの突破から波状攻撃を見せるも、攻撃のスイッチを入れるにはあまりにも遅すぎた。結果、なすすべもなく0－2で完敗し、かつての恩師に勝点3の手土産を渡してしまった。
2,2,0,1-1,Nah
3,1,1,5-2,前節にアウェイで今季初勝利を飾り、ホーム開幕戦を迎えた札幌。守備時には全員が自陣に戻ってボールを奪取し、攻撃ではAロペスを中心に左右にボールを展開。試合を優位に進め、1点リードで折り返す。後半は開始直後から前がかりになった清水に対し、今度はカウンター攻撃で応戦。これで勢いはさらに増し、素早い攻撃から何度もゴールへと迫る。終わってみればAロペスの連続追撃弾で大量5得点を挙げての白星を飾った。一方の清水は、開幕からの3戦で10失点を喫しており、守備の改善が求められる。
4,2,1,0-0,Nah
5,0,1,1-3,清水戦では、鈴木とAロペスの前線2人で5得点を奪った札幌だが、今節の相手は11試合連続勝ちなしと相性の悪い鹿島。積極的な攻撃を展開するが、「試合巧者な鹿島」に2点リードを許し、試合を折り返す。後半開始から徐々に攻撃のリズムを取り戻し、押し込む時間帯が続くも、レアンドロに3点目を奪われてしまう。後半40分にAロペスの2試合連続ゴールで1点を返すが、反撃もここまで。今回も苦汁をなめさせられる結果となった。
6,0,0,4-0,4節での敗戦から8連敗を喫した昨季との違いを見せることに成功した名古屋。速い出足で中盤を支配するほか、パスワークの際の味方との関わり方で相手を圧倒すると、決定機をことごとく沈めて前半だけで3点を奪って折り返す。後半に入っても集中を切らさずに1点を追加する。すると、守備面でも米本を中心に札幌のカウンターをつぶすなど、光るプレーを連発。最後まで統率されたラインで相手の攻撃を抑え、ほぼ完璧な試合運びで今季三度目の完封勝利を達成した。
7,0,1,1-2,前からの激しいプレスと多くの味方の攻撃参加でゴールを目指す札幌と、後方からつなぎつつ、有効なスペースを突いて攻め込む大分。両者のスタイルのぶつかり合いが終始続いた一戦となった。札幌は開始早々に先手を取られると、プレスの意識をさらに強化。だが、これで裏のスペースを空けると、次第に藤本らに突かれて2点目を献上する。大分としては狙い通りの、札幌としては勢いに水を差される1点となった。後半も試合の大勢はほぼ変わらず。ホームチームは90分にわたって攻撃に圧力を見せ続けただけに、悔しい敗戦となった。
8,1,1,4-1,Nah
9,1,0,0-1,両チームともに締まったプレーを見せ、緊張感のある固い内容の試合となる。ホームのC大阪は連係でペナルティエリアへの進入を試みる札幌の攻撃に耐え、仕掛ける際には前への意識が高い片山や松田を中心に右サイドからの打開を狙う。後半に入ってもなお、一進一退の攻防が続くが、セットプレーから進藤に今季初得点を決められ、苦しい展開になる。その後は収まりの良い都倉に縦パスを入れて攻撃を仕掛け続けるも、タイムアップ。復調のけはいを見せていただけに、痛い敗戦を喫してしまった。


In [42]:
for i in range(len(team_name_list)):
    show_dataframe(i)

FileNotFoundError: File b'/Users/gitrepos/toto/trial/\xe5\x8c\x97\xe6\xb5\xb7\xe9\x81\x93\xe3\x82\xb3\xe3\x83\xb3\xe3\x82\xb5\xe3\x83\x89\xe3\x83\xbc\xe3\x83\xac\xe6\x9c\xad\xe5\xb9\x8c.csv' does not exist